In [1]:
import pandas as pd
from pandas import DataFrame

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.head(10)

,ID,cr_ID,assembly_time,eq_ID,process_time,ramp,servo_defect,data_defect,status
0,HDDSN00000,CRSTR01L,2015-03-09 09:36:00,EGRWT0113,2015-03-09 11:33:55,4561.0,6083.0,1341.0,pass
1,HDDSN00001,CRSTR01Q,2015-03-10 02:13:00,EGRWT0102,2015-03-10 03:16:18,3898.0,1819.0,8037.0,pass
2,HDDSN00002,CRSTR01N,2015-03-06 23:06:00,EGRWT0103,2015-03-07 00:00:12,3016.0,4997.0,1583.0,pass
3,HDDSN00003,CRSTR01A,2015-03-08 05:10:00,EGRWT0107,2015-03-08 06:36:18,1605.0,4415.0,3345.0,pass
4,HDDSN00004,CRSTR01C,2015-03-09 06:44:00,EGRWT0113,2015-03-09 09:13:40,1002.0,2394.0,3296.0,pass
5,HDDSN00005,CRSTR01I,2015-03-10 14:10:00,EGRWT0112,2015-03-10 15:15:19,1984.0,2735.0,2953.0,pass
6,HDDSN00006,CRSTR01F,2015-03-08 11:34:00,EGRWT0105,2015-03-08 13:29:41,1356.0,2748.0,2974.0,pass
7,HDDSN00007,CRSTR01D,2015-03-11 23:06:00,EGRWT0105,2015-03-12 00:26:45,1643.0,3566.0,1482.0,pass
8,HDDSN00008,CRSTR01E,2015-03-08 05:29:00,EGRWT0114,2015-03-08 06:07:16,1618.0,3610.0,1823.0,pass
9,HDDSN00009,CRSTR01M,2015-03-09 17:08:00,EGRWT0103,2015-03-09 19:48:12,4789.0,3331.0,7306.0,fail


In [4]:
# 결측값 확인
train_nan_num = train.isnull().sum().sum()
test_nan_num = test.isnull().sum().sum()
print(train_nan_num, test_nan_num)

0 0


#  Preprocessing

### train 전처리

In [5]:
# 1. ID 숫자화 => ID_num
train['ID_num'] = train['ID'].str.slice(start=5)
train['ID_num'] = pd.to_numeric(train['ID_num'])

# 2. cr_ID 숫자화 => cr_ID_num
label = LabelEncoder()
train['cr_ID_num'] = train['cr_ID'].str.slice(start=7)
train['cr_ID_num'] = label.fit_transform(train['cr_ID_num'])

# 3. eq_ID 숫자화 => eq_ID_num
train['eq_ID_num'] = train['eq_ID'].str.slice(start=7)
train['eq_ID_num'] = train['eq_ID_num'].astype(int)

# 4. process_time과 assembly_time 사이 간격 구하기 => period_sec (초단위)
train['assembly_time_datetime'] = train['assembly_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
train['process_time_datetime'] = train['process_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
train['period'] = train['process_time_datetime'] - train['assembly_time_datetime']
train['period_sec'] = train['period'].apply(lambda x : x.seconds)
# 4-1. period_sec 구간 나누기 => period_sec_bin_code
train['period_sec_bin'] = pd.qcut(train['period_sec'].astype(int), 10)
train['period_sec_bin_code'] = label.fit_transform(train['period_sec_bin'])

# 5. ramp 구간 나누기 => ramp_bin_code
train['ramp_bin'] = pd.qcut(train['ramp'].astype(int), 5)
train['ramp_bin_code'] = label.fit_transform(train['ramp_bin'])

# 6. servo_defect 구간 나누기 => servo_defect_bin_code
train['servo_defect_bin'] = pd.qcut(train['servo_defect'].astype(int), 5)
train['servo_defect_bin_code'] = label.fit_transform(train['servo_defect_bin'])

# 7. data_defect 구간 나누기 => data_defect_bin_code
train['data_defect_bin'] = pd.qcut(train['data_defect'].astype(int), 5)
train['data_defect_bin_code'] = label.fit_transform(train['data_defect_bin'])

# 8. assembly_time 일 추출 => assembly_time_date
train['assembly_time_date'] = train['assembly_time'].str.slice(start=8, stop=10).astype(int)
# 8-1. assembly_time 시간 추출 => assembly_time_time
train['assembly_time_time'] = train['assembly_time'].apply(lambda x : int(x[11:13]))
# 8-2. assembly_time 요일 추출 => assembly_time_day
train['assembly_time_day'] = train['assembly_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

# 9. process_time 일 추출 => assembly_time_date
train['process_time_date'] = train['process_time'].str.slice(start=8, stop=10).astype(int)
# 9-1. process_time 시간 추출 => assembly_time_time
train['process_time_time'] = train['process_time'].apply(lambda x : int(x[11:13]))
# 9-2. process_time 요일 추출 => assembly_time_day
train['process_time_day'] = train['process_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

### test 전처리

In [6]:
# 1. ID 숫자화 => ID_num
test['ID_num'] = test['ID'].str.slice(start=5)
test['ID_num'] = pd.to_numeric(test['ID_num'])

# 2. cr_ID 숫자화 => cr_ID_num
test['cr_ID_num'] = test['cr_ID'].str.slice(start=7)
test['cr_ID_num'] = label.fit_transform(test['cr_ID_num'])

# 3. eq_ID 숫자화 => eq_ID_num
test['eq_ID_num'] = test['eq_ID'].str.slice(start=7)
test['eq_ID_num'] = test['eq_ID_num'].astype(int)

# 4. process_time과 assembly_time 사이 간격 구하기 => period_sec (초단위)
test['assembly_time_datetime'] = test['assembly_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
test['process_time_datetime'] = test['process_time'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
test['period'] = test['process_time_datetime'] - test['assembly_time_datetime']
test['period_sec'] = test['period'].apply(lambda x : x.seconds)
# 4-1. period_sec 구간 나누기 => period_sec_bin_code
test['period_sec_bin'] = pd.qcut(test['period_sec'].astype(int), 10)
test['period_sec_bin_code'] = label.fit_transform(test['period_sec_bin'])

# 5. ramp 구간 나누기 => ramp_bin_code
test['ramp_bin'] = pd.qcut(test['ramp'].astype(int), 5)
test['ramp_bin_code'] = label.fit_transform(test['ramp_bin'])

# 6. servo_defect 구간 나누기 => servo_defect_bin_code
test['servo_defect_bin'] = pd.qcut(test['servo_defect'].astype(int), 5)
test['servo_defect_bin_code'] = label.fit_transform(test['servo_defect_bin'])

# 7. data_defect 구간 나누기 => data_defect_bin_code
test['data_defect_bin'] = pd.qcut(test['data_defect'].astype(int), 5)
test['data_defect_bin_code'] = label.fit_transform(test['data_defect_bin'])

# 8. assembly_time 일 추출 => assembly_time_date
test['assembly_time_date'] = test['assembly_time'].str.slice(start=8, stop=10).astype(int)
# 8-1. assembly_time 시간 추출 => assembly_time_time
test['assembly_time_time'] = test['assembly_time'].apply(lambda x : int(x[11:13]))
# 8-2. assembly_time 요일 추출 => assembly_time_day
test['assembly_time_day'] = test['assembly_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

# 9. process_time 일 추출 => assembly_time_date
test['process_time_date'] = test['process_time'].str.slice(start=8, stop=10).astype(int)
# 9-1. process_time 시간 추출 => assembly_time_time
test['process_time_time'] = test['process_time'].apply(lambda x : int(x[11:13]))
# 9-2. process_time 요일 추출 => assembly_time_day
test['process_time_day'] = test['process_time'].apply(lambda x : datetime(int(x[0:4]), int(x[5:7]), int(x[8:10])).weekday())

# 전처리 후 파일 저장

In [7]:
train.to_csv("preprocessed_train.csv", mode='w')
test.to_csv("preprocessed_test.csv", mode='w')